# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [3]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [4]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [5]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [6]:
df_trade_summ.sort_values('feesPaid', ascending=False)

,asset,feesPaid
22,sBTC,1.192329e+06
23,sETH,1.159977e+06
17,OP,2.311247e+05
18,SOL,1.308092e+05
14,LINK,1.196018e+05
15,MATIC,1.082547e+05
7,BNB,7.652804e+04
12,FTM,6.140651e+04
2,ARB,5.770838e+04
5,AVAX,4.882717e+04


## Get the market data

In [7]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [8]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,6.603159e+06
1,sBTC,6.802179e+06
2,LINK,2.485106e+05
3,SOL,4.213846e+05
4,AVAX,2.983335e+05
5,AAVE,3.382963e+03
6,UNI,9.412789e+04
7,MATIC,3.707246e+05
8,APE,8.367129e+04
9,DYDX,3.374078e+04


In [9]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,6.603159e+06,7.689956e+06,1.159977e+06,-2.246775e+06
1,sBTC,6.802179e+06,8.202613e+06,1.192329e+06,-2.592763e+06
2,LINK,2.485106e+05,3.641296e+05,1.196018e+05,-2.352209e+05
3,SOL,4.213846e+05,5.555052e+05,1.308092e+05,-2.649298e+05
4,AVAX,2.983335e+05,2.195172e+05,4.882717e+04,2.998921e+04
5,AAVE,3.382963e+03,1.904010e+03,5.185335e+03,-3.706382e+03
6,UNI,9.412789e+04,1.088740e+05,1.187084e+04,-2.661698e+04
7,MATIC,3.707246e+05,5.131222e+05,1.082547e+05,-2.506523e+05
8,APE,8.367129e+04,5.659213e+04,3.237938e+04,-5.300228e+03
9,DYDX,3.374078e+04,7.082577e+04,4.177234e+04,-7.885734e+04


In [10]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [11]:
df_pnl['netPnl'].sum()

-6562271.118769385

In [12]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
1,sBTC,-2592762.93
0,sETH,-2246774.82
11,OP,-362123.50
3,SOL,-264929.79
7,MATIC,-250652.29
2,LINK,-235220.88
19,FTM,-164432.40
23,ARB,-143831.74
10,BNB,-121745.28
9,DYDX,-78857.34
